<a href="https://www.kaggle.com/code/jhoaneusse/international-debt-bq?scriptVersionId=145495430" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In this nootebook we analyze international debt statistics answering the questions contained on the following datacamp project:  
https://www.datacamp.com/projects/754

Using the BigQuery dataset *world_bank_intl_debt*

In [1]:
from google.cloud import bigquery

In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "world-bank-intl-debt" dataset
dataset_ref = client.dataset("world_bank_intl_debt", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exploration of the dataset

In [3]:
# List all the tables in the "world-bank-intl-debt" dataset
tables = list(client.list_tables(dataset))

tables_id=[table.table_id for table in tables]

tables_id

['country_series_definitions',
 'country_summary',
 'international_debt',
 'series_summary',
 'series_times']

In [4]:
# Construct a reference to the "international_debt" table
table_ref = dataset_ref.table("international_debt")

# API request - fetch the table
table = client.get_table(table_ref)

In [5]:
# Print the schema of the "international_debt" table 
table.schema

[SchemaField('country_name', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('country_code', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('indicator_name', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('indicator_code', 'STRING', 'NULLABLE', '', (), None),
 SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None),
 SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None)]

In [6]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Chad,TCD,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN
1,Chad,TCD,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN
2,Chad,TCD,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN
3,Chad,TCD,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN
4,Chad,TCD,GNI (current US$),NY.GNP.MKTP.CD,NaN,NaN


Printing all the distinct names of countries 

In [7]:
# Query to count the extract the rows in country_name column
country_query = """
           SELECT country_name AS country 
               FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
               GROUP BY country
       """


# Set up the query
cq_job = client.query(country_query)

# API request - run the query, and return a pandas DataFrame
countries = cq_job.to_dataframe()

#Extract the number
print(countries.to_string())

                                                country
0                                                  Chad
1                                                  Fiji
2                                                  Mali
3                                                  Peru
4                                                  Togo
5                                                 Benin
6                                                 China
7                                                 Gabon
8                                                 Ghana
9                                                 Haiti
10                                                India
11                                                Kenya
12                                                Nepal
13                                                Niger
14                                                Samoa
15                                                Sudan
16                                              

We see that there are rows containing info that doesn't match with a particular country but a bunch of countries:

* Low & middle income
* Lower middle income
* Upper middle income
* Sub-Saharan Africa (excluding high income)
* East Asia & Pacific (excluding high income)
* Least developed countries: UN classification
* Europe & Central Asia (excluding high income)
* Latin America & Caribbean (excluding high income)
* Middle East & North Africa (excluding high income)
* South Asia 
* Middle income
* Upper middle income


We're going to ignore this rows in the following querys using the query sentence: WHERE (country_name NOT LIKE '%income%' AND country_name NOT LIKE 'South Asia' AND country_name NOT LIKE '%UN classification')

# What is the number of distinct countries in the dataset?

In [8]:
# Query to count the distinct rows in country_name column
num_country_query = """
           SELECT COUNT(DISTINCT country_name) AS num_country
               FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
       """


# Set up the query
ncq_job = client.query(num_country_query)

# API request - run the query, and return a pandas DataFrame
num_country = ncq_job.to_dataframe()

#Extract the number of countries (substracting the 12 rows of non-countries)
num_country.loc[0,'num_country']-12

123

# Which are the distinct debt indicators?

In [9]:
# Query to extract the distinct rows in indicator_name column
deb_indic_query = """
           SELECT indicator_name AS debt_indicator, indicator_code AS debt_indicator_id
               FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
               GROUP BY debt_indicator, debt_indicator_id  
       """


# Set up the query
diq_job = client.query(deb_indic_query)

# API request - run the query, and return a pandas DataFrame
deb_indic = diq_job.to_dataframe()

#Extracting the number
deb_indic.head()

,debt_indicator,debt_indicator_id
0,GNI (current US$),NY.GNP.MKTP.CD
1,IDA grants (current US$),DT.DIS.IDAG.CD
2,Debt buyback (current US$),DT.DSB.DPPG.CD
3,"PPG, IDA (AMT, current US$)",DT.AMT.MIDA.CD
4,"PPG, IDA (DIS, current US$)",DT.DIS.MIDA.CD


# What is  the total amount of debt owed by each country in million of dollars?

In [10]:
total_debt_query = """
           SELECT country_name,
               SUM(value)/ 1000000 AS total_debt_MUSD 
               FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
               WHERE (country_name NOT LIKE '%income%' AND country_name NOT LIKE 'South Asia' AND country_name NOT LIKE '%UN classification')
               GROUP BY country_name
               ORDER BY total_debt_MUSD DESC 
       """


# Set up the query
tdq_job = client.query(total_debt_query)

# API request - run the query, and return a pandas DataFrame
total_debt = tdq_job.to_dataframe()

#Extract the number
total_debt.head()

,country_name,total_debt_MUSD
0,Mexico,6.261206e+06
1,IDA only,5.934219e+06
2,India,4.476825e+06
3,Brazil,3.859451e+06
4,Russian Federation,3.622486e+06


According to this dataset the country with more debt is Mexico with aprox **6.2 million million USD**

# What is the total amount of debt owed by all countries?

(without including the groups of countries described above)

In [11]:
debt_query = """
           SELECT
               SUM(value)/ 1000000 AS total_debt_MUSD 
               FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
               WHERE (country_name NOT LIKE '%income%' AND country_name NOT LIKE 'South Asia' AND country_name NOT LIKE '%UN classification')
       """


# Set up the query
dq_job = client.query(debt_query)

# API request - run the query, and return a pandas DataFrame
debt = dq_job.to_dataframe()

#Extract the number
debt.head()

,total_debt_MUSD
0,6.744104e+07


# What is the average amount of debt across indicators?

In [12]:
AvgDbtInd_query = """
                            SELECT  indicator_code AS debt_indicator_id,
                               indicator_name AS debt_indicator, 
                            AVG(value)/1000000 as avgDebtPerIndicator_MUSD
                            FROM `bigquery-public-data.world_bank_intl_debt.international_debt` 
                            GROUP BY indicator_code, indicator_name
                            ORDER BY avgDebtPerIndicator_MUSD DESC  
                    """


# Set up the query
adiq_job = client.query(AvgDbtInd_query)

# API request - run the query, and return a pandas DataFrame
AvgDbtInd = adiq_job.to_dataframe()

#Extract the number
AvgDbtInd.head()

,debt_indicator_id,debt_indicator,avgDebtPerIndicator_MUSD
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",10084.981900
1,DT.AMT.DPPG.CD,"Principal repayments on external debt, public ...",6372.337363
2,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",3712.644537
3,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",3221.343287
4,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",3150.994076
